In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from collections import defaultdict
from textwrap import wrap

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

import transformers
#from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import BertModel, AdamW, get_linear_schedule_with_warmup
from transformers import BertForSequenceClassification, BertTokenizer

import torch
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader


In [ ]:
PRE_TRAINED_MODEL_NAME = 'bert-base-cased'

class SentimentClassifier(nn.Module):

  def __init__(self, n_classes):
    super(SentimentClassifier, self).__init__()
    self.bert = BertModel.from_pretrained(PRE_TRAINED_MODEL_NAME, return_dict = False)
    self.drop = nn.Dropout(p=0.3)
    self.out = nn.Linear(self.bert.config.hidden_size, n_classes)
  
  def forward(self, input_ids, attention_mask):
    _, pooled_output = self.bert(
      input_ids=input_ids,
      attention_mask=attention_mask
    )
    output = self.drop(pooled_output)
    return self.out(output)

In [ ]:
# class_names = ['negative', 'neutral', 'positive']
class_names = ['neutral', 'negative', 'positive']

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model = SentimentClassifier(len(class_names))
model.load_state_dict(torch.load('best_model_state.bin'))
model = model.to(device)

In [ ]:
review_texts = ["I love completing my todos! Greatest app ever!!!","bad hate"]
MAX_LEN = 160
tokenizer = BertTokenizer.from_pretrained(PRE_TRAINED_MODEL_NAME)

def infer_model(review_texts):
    res = []
    for review_text in review_texts:
        encoded_review = tokenizer.encode_plus(
        review_text,
        max_length=MAX_LEN,
        add_special_tokens=True,
        return_token_type_ids=False,
        pad_to_max_length=True,
        return_attention_mask=True,
        return_tensors='pt',
        )
        input_ids = encoded_review['input_ids'].to(device)
        attention_mask = encoded_review['attention_mask'].to(device)

        output = model(input_ids, attention_mask)
        _, prediction = torch.max(output, dim=1)

        res.append(class_names[prediction])
    return res

predictions = infer_model(review_texts)

for i, review_text in enumerate(review_texts):
    print(f'{predictions[i]} : {review_text}')


# OLD STUFF

In [ ]:
# bert_model = BertModel.from_pretrained('bert-base-cased')
# bert_model.load_state_dict(torch.load('best_model_state.bin'))
# bert_model.eval()



In [ ]:
# tokenizer = BertTokenizer.from_pretrained("bert-base-cased")
# tokens = tokenizer.encode("I haaaaaaated this, absolutely the WORST. 🙄", return_tensors='pt')
# result = bert_model(tokens)
# print(result)
# print(result.logits)
# print(torch.argmax(result.logits))

In [ ]:
# bert_model = BertModel.from_pretrained('best_model_state.bin', cache_dir=None, from_tf=False, state_dict=None)
# bert_model.eval()

In [ ]:
# # import torch
# # from transformers import *
# tokenizer = BertTokenizer.from_pretrained('./')
# config = BertConfig('./best_model_config.json')
# model = BertModel(config)
# model.load_state_dict(torch.load('./best_model_state.bin', map_location=torch.device('cpu')))
# model.eval()

In [ ]:
# model = SentimentClassifier(n_classes=3)
# model.load_state_dict(torch.load(PATH))
# model.eval()